In [0]:
# imports
from utils.api import buscar_dados, obter_estado_id_por_uf
from pyspark.sql.functions import lit, current_timestamp
import time

In [0]:
#Variaveis do Unity Catalog
CATALOGO = "agroclima"
SCHEMA = "bronze"
TABELA = "temperatura"
TABELA_LOCALIZACAO_MUNICIPIOS = "localizacao_municipios"

# Montar caminho da tabela
caminho_tabela = f"{CATALOGO}.{SCHEMA}.{TABELA}"
caminho_tabela_localizacao = f"{CATALOGO}.{SCHEMA}.{TABELA_LOCALIZACAO_MUNICIPIOS}"


In [0]:

#Parametros para facilitar a execução e reutilização do notebook para carga de diferentes anos e estados
dbutils.widgets.text("anos_param", "2023,2022", "Lista de Anos")
dbutils.widgets.text("uf", "MG", "UF")

In [0]:
#Busca Dados de Localização dos Municipios
URL_LOCALIZACAO_MUNICIPIOS = "https://raw.githubusercontent.com/kelvins/municipios-brasileiros/main/json/municipios.json"

#Executa a API com a URL para buscar Municipios com Localização 
dados_municipios = buscar_dados(URL_LOCALIZACAO_MUNICIPIOS, encoding='utf-8-sig')

# Cria Dataframe com Dados de Municipios
df_localizacao_municipios = spark.createDataFrame(dados_municipios)

In [0]:
# Transforma os parametros em variaveis
lista_anos = dbutils.widgets.get('anos_param').split(",")
uf = dbutils.widgets.get('uf')

# Seta Variavel com o Id da UF 
estado_id = obter_estado_id_por_uf(uf)

In [0]:
#Filtra Municipios da UF passada no parametro
df_uf = df_localizacao_municipios.filter(df_localizacao_municipios['codigo_uf']  == estado_id)

In [0]:
# Função que obtem os dados de temperatura de um municipio
def obter_temperaturas(ano, lat, lon, municipio_id):
    
    params = {
       "latitude": f"{lat}",
       "longitude": f"{lon}",
       "daily": "rain_sum,temperature_2m_mean,relative_humidity_2m_mean",
       "timezone": "America/Sao_Paulo",
       "start_date": f"{ano}-01-01",
       "end_date": f"{ano}-12-31"

    }

    URL_TEMPERATURA = f"https://archive-api.open-meteo.com/v1/archive"
   
    #Executa a API com a URL personalizada por municipio
    dados_temperaturas = buscar_dados(URL_TEMPERATURA, params)

    #adiciona Id do municipio ao json
    dados_temperaturas['municipio_id'] = municipio_id

    #adiciona ano ao json
    dados_temperaturas['ano'] = ano
    
    return dados_temperaturas

    

In [0]:
# Lista para armazenar dados 
lista_temperaturas_por_municipio = []

# Iteração nos Anos passado no parametro
for ano in lista_anos:

    # Iteração na lista de municipios
    for municipio in df_uf.collect():
        try:

            #Executa a função que retorna os dados de Temperatura por municipio
            temperatura_municipio = obter_temperaturas(ano, municipio['latitude'], municipio['longitude'], municipio['codigo_ibge'])

            #Salva dados do municipio na Lista 
            lista_temperaturas_por_municipio.append(temperatura_municipio)

        except Exception as e:
            pass

# Transforma a lista em um DataFrame
df_temperaturas = spark.createDataFrame(lista_temperaturas_por_municipio)
#display(df_final)

In [0]:
# Adiciona UF no DataFrame e Data de Ingestão
df_temperaturas = df_temperaturas.withColumn("uf", lit(uf)) \
                                  .withColumn("_data_ingestao", current_timestamp())


In [0]:
#Salva Dados de Teperatura sobreescrevendo apenas dados dos anos e uf passados em parametro
df_temperaturas.write \
        .format("delta") \
        .mode("overwrite") \
        .partitionBy("uf", "ano") \
        .option('replaceWhere', f"ano in ({','.join(lista_anos)}) and uf = '{uf}'") \
        .option("mergeSchema", "true") \
        .saveAsTable(caminho_tabela)

In [0]:
# Adiciona Data de Ingestão
df_localizacao_municipios = df_localizacao_municipios.withColumn("_data_ingestao", current_timestamp())

#Salva Dataframe como uma tabela Delta
df_localizacao_municipios.write \
                            .format("delta") \
                            .mode("overwrite") \
                            .partitionBy("codigo_uf") \
                            .option("mergeSchema", "true") \
                            .saveAsTable(caminho_tabela_localizacao)